In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-MiniLM-L6-cos-v5'
bi_encoder = SentenceTransformer(model_name, device=device)
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens

In [3]:
docs = pd.read_table('msmarco-docs.tsv', header=None, names=['id', 'url', 'title', 'body'], usecols=['body'], dtype='string')['body']

In [4]:
docs.dropna(inplace=True)

In [5]:
# docs.index.to_series().to_csv('corpus_id_to_doc_id.txt', header=False, index=False)

In [6]:
docs.map(type).value_counts()

body
<class 'str'>    3201821
Name: count, dtype: int64

In [7]:
#Encode all passages
docs = docs.tolist()

In [8]:
corpus_embeddings = bi_encoder.encode(docs, convert_to_tensor=True, show_progress_bar=True, device=device)

Batches:   0%|          | 0/100057 [00:00<?, ?it/s]

In [9]:
# save embeddings
torch.save(corpus_embeddings, f'corpus_embeddings_{model_name}.pt')